In [1]:
# !pip install pandas
# !pip install numpy
# !pip install pandas_datareader
# !pip install statsmodels
# !pip install pmdarima
# !pip install fastapi
# !pip install uvicorn
# !pip install pydantic

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import pandas_datareader as pdr
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from fastapi import FastAPI
import uvicorn
import pydantic


def data(ticker):
    
    df = pdr.DataReader(ticker, data_source='yahoo', start='2016-01-01')
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df = pd.Series(df['Close'])
    last_day=df[-1]
    return df , last_day

def best_order(df):
    
    model = pm.auto_arima(df, start_p=0, start_q=0, test='adf', max_p=2, max_q=2, m=1,d=None,seasonal=False   
                      ,start_P=0,D=0, trace=True,error_action='ignore',suppress_warnings=True,stepwise=True)
    order = model.order
    return order

def model(df,order,last_day):
    model = ARIMA(df, order=order)
    model_fit = model.fit(disp=0)
    fc ,se, conf = model_fit.forecast(1)
    diff = fc - last_day
    return fc , diff

def overall(ticker):
    df,last_day = data(ticker)
    order  = best_order(df)
    fc , diff =model(df , order ,last_day)
    return last_day , fc , diff 





app = FastAPI()

@app.get('/')

def index():
    return {'message': 'Hello!'}

@app.post('/predict')

async def predict_price(ticker:str):
    
    last_day , fc , diff = overall(ticker)
    return {'last_price':last_day,'prediction':fc[0],'difference:':diff[0]}
       
if __name__ == '__main__':
    uvicorn.run(app, host='127.0.0.1', port=6064)
    
    

RuntimeError: This event loop is already running

INFO:     Started server process [12428]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:6064 (Press CTRL+C to quit)


INFO:     127.0.0.1:54684 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:54684 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54683 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:54683 - "GET /openapi.json HTTP/1.1" 200 OK
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-591.968, Time=1.38 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-590.185, Time=0.15 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-590.187, Time=0.42 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-593.849, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-588.246, Time=0.49 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 2.495 seconds
INFO:     127.0.0.1:54685 - "POST /predict?ticker=F HTTP/1.1" 200 OK


C:\Users\Admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\Admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
